In [46]:
#Ref
import pandas as pd
import glob
import numpy as np
import plotly.express as px
path = '/Users/jiakai/Desktop/SURF/code/_spirit/'
n_cycles = 120


# Pattern to match all relevant CSV files
file_pattern = path + "Susceptibility_multi_gammas_10_120_per_gamma_20_anisotropy_0.7_gamma_0.0001_ref*.csv"

# List all matching files
csv_files = glob.glob(file_pattern)

# Print how many files were found
print(f"Found {len(csv_files)} files.")
repeats = len(csv_files)

for f in csv_files:
    print(f)

# Read and concatenate all files
df_all = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

df_avg = (
        df_all.groupby(["gamma", "Ht", "direction"], as_index=False)
        .agg(
            chi_mean=("chi", "mean"),
            chi_std=("chi", lambda x: x.std(ddof=1) / np.sqrt(n_cycles*repeats))
        )
    )

fig_ref = px.line(
    df_avg,
    x="Ht",
    y="chi_mean",
    error_y="chi_std",
    color="direction",  # Different color for up/down
    line_dash="gamma",  # Optional: if you vary gamma too
    markers=True,
    labels={
        "Ht": "Ht (T)",
        "chi_mean": "Susceptibility χ",
        "direction": "Sweep Direction",
    },
    title="Susceptibility χ vs Ht (sweep up vs down)",
    template="plotly_white"
)

fig_ref.show()







Found 3 files.
/Users/jiakai/Desktop/SURF/code/_spirit/Susceptibility_multi_gammas_10_120_per_gamma_20_anisotropy_0.7_gamma_0.0001_ref_2.csv
/Users/jiakai/Desktop/SURF/code/_spirit/Susceptibility_multi_gammas_10_120_per_gamma_20_anisotropy_0.7_gamma_0.0001_ref_1.csv
/Users/jiakai/Desktop/SURF/code/_spirit/Susceptibility_multi_gammas_10_120_per_gamma_20_anisotropy_0.7_gamma_0.0001_ref.csv


In [47]:
#Relaxed
import pandas as pd
import glob
import numpy as np
import plotly.express as px
path = '/Users/jiakai/Desktop/SURF/code/_spirit/'
n_cycles = 240


# Pattern to match all relevant CSV files
file_pattern = path + "Susceptibility_multi_gammas_10_240_per_gamma_20_anisotropy_0.7_relax_step_400_gammas_-5_relax_0.8_gamma_0.0001_relaxed*.csv"

# List all matching files
csv_files = glob.glob(file_pattern)

# Print how many files were found
print(f"Found {len(csv_files)} files.")
repeats = len(csv_files)
for f in csv_files:
    print(f)

# Read and concatenate all files
df_all_relaxed = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Average over cycles, std divided by sqrt(n_cycles)
df_avg_relaxed = (
    df_all_relaxed.groupby(["gamma", "Ht"], as_index=False)
    .agg(
        chi_mean=("chi", "mean"),
        chi_std=("chi", lambda x: x.std(ddof=1) / np.sqrt(n_cycles*repeats))
    )
)

# Plot
fig_relaxed = px.line(
    df_avg_relaxed,
    x="Ht",
    y="chi_mean",
    error_y="chi_std",
    color="gamma",  # Different color per gamma
    markers=True,
    labels={
        "Ht": "Ht (T)",
        "chi_mean": "Susceptibility χ",
        "gamma": "Gamma",
    },
    title="Susceptibility χ vs Ht for different Γ (gamma)",
    template="plotly_white"
)


fig_relaxed.show()






Found 2 files.
/Users/jiakai/Desktop/SURF/code/_spirit/Susceptibility_multi_gammas_10_240_per_gamma_20_anisotropy_0.7_relax_step_400_gammas_-5_relax_0.8_gamma_0.0001_relaxed.csv
/Users/jiakai/Desktop/SURF/code/_spirit/Susceptibility_multi_gammas_10_240_per_gamma_20_anisotropy_0.7_relax_step_400_gammas_-5_relax_0.8_gamma_0.0001_relaxed_1.csv


In [ ]:
#Ht = H_relax before relaxation
path = '/Users/jiakai/Desktop/SURF/code/_spirit/'
n_cycles = 240

df = pd.read_csv("chi_before_relax_10_240_per_gamma_20_anisotropy_0.7_relax_step_400_gammas_-5_relax_0.8_gamma_0.0001*.csv")
# List all matching files
csv_files = glob.glob(file_pattern)

# Print how many files were found
print(f"Found {len(csv_files)} files.")
repeats = len(csv_files)
for f in csv_files:
    print(f)


# Read and concatenate all files
df_all_no_relax = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)

# Average over cycles, std divided by sqrt(n_cycles)
df_avg_no_relax = (
    df_all_no_relax.groupby(["gamma", "Ht"], as_index=False)
    .agg(
        chi_mean=("chi", "mean"),
        chi_std=("chi", lambda x: x.std(ddof=1) / np.sqrt(n_cycles*repeats))
    )
)

# Plot
fig_no_relax= px.line(
    df_avg_no_relax,
    x="Ht",
    y="chi_mean",
    error_y="chi_std",
    color="gamma",  # Different color per gamma
    markers=True,
    labels={
        "Ht": "Ht (T)",
        "chi_mean": "Susceptibility χ",
        "gamma": "Gamma",
    },
    title="Susceptibility χ vs Ht for different Γ (gamma)",
    template="plotly_white"
)


fig_no_relax.show()

In [48]:
import plotly.graph_objects as go

# Create a new figure
fig = go.Figure()

# --- Plot relaxed data ---
for gamma, df_group in df_avg_relaxed.groupby("gamma"):
    fig.add_trace(
        go.Scatter(
            x=df_group["Ht"],
            y=df_group["chi_mean"],
            error_y=dict(type="data", array=df_group["chi_std"]),
            mode="lines+markers",
            name=f"Relaxed γ={gamma}",
            line=dict(dash="solid"),
            marker=dict(symbol="circle"),
        )
    )

# --- Plot Ht before relaxation data ---
for gamma, df_group in df_avg_no_relax.groupby("gamma"):
    fig.add_trace(
        go.Scatter(
            x=df_group["Ht"],
            y=df_group["chi_mean"],
            error_y=dict(type="data", array=df_group["chi_std"]),
            mode="lines+markers",
            name=f"Relaxed γ={gamma}",
            line=dict(dash="solid"),
            marker=dict(symbol="circle"),
        )
    )

# --- Plot sweep data (with direction) ---
for (direction, gamma), df_group in df_avg.groupby(["direction", "gamma"]):
    fig.add_trace(
        go.Scatter(
            x=df_group["Ht"],
            y=df_group["chi_mean"],
            error_y=dict(type="data", array=df_group["chi_std"]),
            mode="lines+markers",
            name=f"{direction} γ={gamma}",
            line=dict(dash="solid"),
            marker=dict(symbol="square"),
        )
    )

# Final layout
fig.update_layout(
    title="Susceptibility χ vs Ht (Relaxed and Sweeps)",
    xaxis_title="Ht (T)",
    yaxis_title="Susceptibility χ",
    template="plotly_white",
    legend_title="Dataset",
)

fig.write_html(path+'susceptibility.html')


In [49]:
fig.show()

In [7]:
dim = 10
import os
import numpy as np
fn = "dipolar_arr"
prefix = "DDI_exp_14_G0p00005_Ht10p0"

base_path = f"dipolar_interaction_matrices_reordered/{dim}_{dim}_{dim}/"
DDI_interaction_x = np.load(os.path.join(base_path, fn + "_x.npy"))

In [8]:
# Temporarily set print options to show full array
np.set_printoptions(threshold=100)

print(DDI_interaction_x)

# (Optional) Reset after if you want default behavior
np.set_printoptions(threshold=1000)


[[-9.17551347e+01 -2.29204738e+02  8.61675890e-18 ...  2.91484002e+01
   2.79119870e+01  2.84712149e+01]
 [-2.29204738e+02 -1.84383038e+03 -1.53967083e-18 ...  4.18325333e+01
   2.84712149e+01  1.96556639e+01]
 [-1.24492789e-17 -1.53967083e-18 -1.84383038e+03 ...  2.84712149e+01
   9.83685651e+00  6.35088572e+00]
 ...
 [ 4.03267000e+00  9.83685651e+00  2.84712149e+01 ... -1.84383038e+03
  -1.53967083e-18  1.14580959e-17]
 [ 1.11782249e+01  2.84712149e+01  4.18325333e+01 ... -1.53967083e-18
  -1.84383038e+03 -2.29204738e+02]
 [ 2.84712149e+01  8.29122471e+01  8.87257465e+01 ... -1.00785014e-17
  -2.29204738e+02 -9.17551347e+01]]


In [14]:
import numpy as np
from scipy import sparse

# Threshold
threshold = 1e-4

# Total number of entries
total_entries = DDI_interaction_x.size

# Count how many will be set to zero
num_below_threshold = np.sum(np.abs(DDI_interaction_x) < threshold)

# Create sparse matrix
DDI_interaction_x_sparse = sparse.csr_matrix(np.where(
    np.abs(DDI_interaction_x) < threshold, 0, DDI_interaction_x
))

# Percentage of entries set to 0
percent_zeroed = 100 * num_below_threshold / total_entries

# Percentage of entries remaining non-zero
percent_nonzero = 100 * DDI_interaction_x_sparse.nnz / total_entries

print(f"Total entries: {total_entries}")
print(f"Entries below threshold: {num_below_threshold} ({percent_zeroed:.2f}%)")
print(f"Remaining non-zero entries: {DDI_interaction_x_sparse.nnz} ({percent_nonzero:.2f}%)")

Total entries: 16000000
Entries below threshold: 2240000 (14.00%)
Remaining non-zero entries: 13760000 (86.00%)
